In [1]:
import xarray as xr
import cf_xarray as cfxr
import numpy as np

import sys

sys.path.append("..")

from cge_modeling.gams.gams_constants import ENERGY, energy_sam
from cge_modeling.gams.from_excel import make_code_dicts

In [2]:
group_order = [
    "Imported Commodities",
    "Domestic Commodities",
    "Activities",
    "Factors",
    "Institution",
    "Import Sales Tax",
    "Domestic Sales Tax",
    "Factor Tax",
    "Factor Subsidity",
    "Tax",
    "Import Margin",
    "Export Margin",
    "Trade",
    "Other",
]

reduced_group_order = [
    "Imported Commodities",
    "Domestic Commodities",
    "Activities",
    "Factors",
    "Institution",
    "Import Sales Tax",
    "Domestic Sales Tax",
    "Factor Tax",
    "Factor Subsidity",
    "Tax",
    "Trade",
]

In [3]:
df = xr.load_dataset("GTAP_SAM_with_errors.nc")
df = cfxr.decode_compress_to_multi_index(df, ["row", "column"]).SAM

In [4]:
path = "data/GTAP_raw_data.xlsx"
code_dicts = make_code_dicts(path)

# Write to Excel

In [5]:
import pandas as pd
import os


def create_excel(df, code_dicts, output_path="data/gtap_SAM_2017.xlsx", force_rewrite=False):

    code_to_country = code_dicts["country"]
    country_to_code = {v: k for k, v in code_to_country.items()}

    country_names = sorted(list(code_to_country.values()))
    country_codes = [country_to_code.get(country) for country in country_names]

    if os.path.isfile(output_path) and not force_rewrite:
        return

    with pd.ExcelWriter("data/gtap_SAM_2017.xlsx", engine="openpyxl") as writer:
        for country in country_codes:
            temp = df.sel(country=country).to_dataframe()["SAM"].unstack(["col_group", "col_value"])
            temp.columns.names = ["group", "value"]
            temp.index.names = ["group", "value"]
            temp = temp.loc[group_order, group_order]

            long_name = code_to_country[country]
            temp.to_excel(writer, sheet_name=long_name[:30])


create_excel(df, code_dicts)

# Single Country 

In [6]:
SRB = df.sel(country="SRB").to_dataframe()["SAM"].unstack(["col_group", "col_value"])
SRB.columns.names = ["group", "value"]
SRB.index.names = ["group", "value"]

In [7]:
def sam_errors(df):
    return np.abs(df.sum(axis=0) - df.sum(axis=1))


def max_error(df, func="max"):
    if func == "max":
        return sam_errors(df).max()
    elif func == "idxmax":
        return sam_errors(df).idxmax()

In [8]:
def merge_values(df, level_0, to_merge, agg_name):
    df = df.copy()
    indexes = [(level_0,) + (x,) for x in to_merge]
    agg_name_temp = f"__{agg_name}__"

    row = df.loc[indexes, :].sum(axis=0)
    df.loc[(level_0, agg_name_temp), :] = row
    df.drop(index=indexes, inplace=True)
    df.sort_index(axis=0, level=[0, 1], inplace=True)

    col = df.loc[:, indexes].sum(axis=1)
    df.loc[:, (level_0, agg_name_temp)] = col
    df.drop(columns=indexes, inplace=True)
    df.sort_index(axis=1, level=[0, 1], inplace=True)

    df.rename(index={agg_name_temp: agg_name}, inplace=True)
    df.rename(columns={agg_name_temp: agg_name}, inplace=True)

    return df


def merge_level_0(df, merge_into, to_merge):
    df = df.copy()
    temp_row = None
    temp_col = None
    for level in to_merge:
        row = df.loc[level, :].sum(axis=0)
        temp_row = temp_row + row if temp_row is not None else row
        df.drop(index=level, inplace=True)

        col = df.loc[:, level].sum(axis=1)
        temp_col = temp_col + col if temp_col is not None else col
        df.drop(columns=level, inplace=True)

    df.loc[merge_into, :] += temp_row
    df.loc[:, merge_into] += temp_col
    return df

In [139]:
def aggregate_SAM(df, to_3x3=False, add_grid=True):
    df = df.copy()
    factors = df.loc["Factors"].index
    capital, land, res, *labor = factors
    for group in ["Factors", "Factor Tax", "Factor Subsidity"]:
        # Combine labor factors into a single factor
        df = merge_values(df, group, labor, "Labor")

        # Combine land and natural resources into capital
        df = merge_values(df, group, [capital, land, res], "Capital")

    #     Merge similar power types
    merges = [
        ("GasP", ("Gas power baseload", "Gas power peakload")),
        ("HydroP", ("Hydro power baseload", "Hydro power peakload")),
        ("OilP", ("Oil power baseload", "Oil power peakload")),
    ]
    for target, sources in merges:
        code_to_source = code_dicts["energy"]
        source_to_code = {v: k for k, v in code_to_source.items()}
        sources = list(map(source_to_code.get, sources))
        for group in [
            "Domestic Commodities",
            "Imported Commodities",
            "Activities",
            "Domestic Sales Tax",
            "Import Sales Tax",
        ]:
            df = merge_values(df, group, sources, target)

    # Rename other power suppliers
    power_rename = {
        "NuclearBL": "NuclearP",
        "CoalBL": "CoalP",
        "OtherBL": "OtherP",
        "WindBL": "WindP",
    }
    df.rename(index=power_rename, inplace=True)
    df.rename(columns=power_rename, inplace=True)

    # Fold margins into the broader import/export accounts
    df = merge_level_0(df, ("Trade", "Rest of World"), ["Import Margin", "Export Margin"])

    # Eliminate the regional household.
    # The market clearing for the regional household is:
    # factor_income + tax_rev = Y_H + Y_G + S

    hh_income = df.loc[("Institution", "Regional Household"), ["Factors"]]
    C_H = df.loc[("Institution", "Private Household"), ("Institution", "Regional Household")].item()

    tax_idx = ["Domestic Sales Tax", "Import Sales Tax", "Tax", "Factor Tax", "Factor Subsidity"]
    tax_income = df.loc[("Institution", "Regional Household"), tax_idx]
    C_G = df.loc[("Institution", "Government"), ("Institution", "Regional Household")].item()

    S = df.loc[("Institution", "Investment"), ("Institution", "Regional Household")].item()
    K_d = df.loc[("Institution", "Regional Household"), ("Other", "Capital Depreciation")].item()

    # Drop the regional household
    df = df.drop(columns=[("Institution", "Regional Household")]).drop(
        index=[("Institution", "Regional Household")]
    )

    # Drop the capital depreciaton as well
    df = df.drop(columns=[("Other", "Capital Depreciation")]).drop(
        index=[("Other", "Capital Depreciation")]
    )

    # We want to split this into two equations:
    # factor_income = Y_H + S_H
    # tax_rev = Y_G + S_G
    # With S - K_d = S_H + S_G

    #     # Household income comes from the factors
    df.loc[("Institution", "Private Household"), ["Factors"]] = hh_income

    #     # Choose S_H so that the household must have a balanced budget, S_H = factor_income - Y_H
    S_HH = hh_income.sum() - C_H
    df.loc[("Institution", "Investment"), ("Institution", "Private Household")] = S_HH

    #     # Tax revenue belongs to the government
    df.loc[("Institution", "Government"), tax_idx] = tax_income

    #     # For goverment savings, set it to whatever respects S + K_d = S_H + S_G
    S_G = S - K_d - S_HH
    df.loc[("Institution", "Investment"), ("Institution", "Government")] = S_G

    df.rename(index={"Private Household": "Household"}, inplace=True)
    df.rename(columns={"Private Household": "Household"}, inplace=True)
    # Next I want to "chop off" the value chain matrix to construct the grid.
    # I need to sum up all electricity-related purchases by all agents, and add them to the "Grid"
    # TnD is Electricity transmission and distribution, plus all the power plants themselves
    power_suppliers = [
        "CoalP",
        "GasP",
        "HydroP",
        "NuclearP",
        "OilP",
        "OtherP",
        "SolarP",
        "WindP",
        "TnD",
    ]

    # Create a new institution, the "grid"
    df.loc[("Institution", "Grid"), :] = 0.0
    df.loc[:, ("Institution", "Grid")] = 0.0

    # Create a new factor, "electricity"
    df.loc[("Factors", "Electricity"), :] = 0.0
    df.loc[:, ("Factors", "Electricity")] = 0.0

    # Create new factor tax and subsidy for electricity
    df.loc[("Factor Tax", "Electricity"), :] = 0.0
    df.loc[:, ("Factor Tax", "Electricity")] = 0.0

    df.fillna(0, inplace=True)

    if add_grid:
        old_df = df.copy()
        for group in ["Domestic", "Imported"]:
            power_slice = [(f"{group} Commodities",) + (x,) for x in power_suppliers]
            tax_slice = [(f'{group.replace("ed", "")} Sales Tax',) + (x,) for x in power_suppliers]
            electrictiy_demand = df.loc[power_slice, :].sum(axis=0)
            electricity_taxes = df.loc[tax_slice, :].sum(axis=0)

            electricity_supply = df.loc[:, power_slice].sum(axis=0)
            electricity_tax_income = df.loc[:, tax_slice].sum(axis=0)

            # Add up all demand for electricity by all agents and make it a factor called 'electricity'
            df.loc[("Factors", "Electricity"), :] += electrictiy_demand

            # Set the direct demands to the powerplants to zero
            df.loc[power_slice, :] = 0.0
            df.loc[tax_slice, :] = 0.0

            # The electricity factor is supplied by the grid
            df.loc[("Institution", "Grid"), ("Factors", "Electricity")] += electrictiy_demand.sum()

            # The grid buys all output from the domestic power plants
            df.loc[power_slice, ("Institution", "Grid")] = electricity_supply

            # Tax burden on all those payments are shifted to the end-users as a factor tax.
            # Reclassify the government income as well
            df.loc[("Factor Tax", "Electricity"), :] += electricity_taxes
            df.loc[("Institution", "Government"), tax_slice] -= electricity_tax_income
            df.loc[
                ("Institution", "Government"), ("Factor Tax", "Electricity")
            ] += electricity_taxes.sum()

        # Don't want to do international trade in electricity factor, only in raw power outputs
        # Reset their electricity purchases to zero
        exports_of_energy = df.loc[("Factors", "Electricity"), ("Trade", "Rest of World")]
        df.loc[("Institution", "Grid"), ("Factors", "Electricity")] -= exports_of_energy
        df.loc[("Factors", "Electricity"), ("Trade", "Rest of World")] = 0.0

        # Restore exports of raw power outputs
        power_slice = [("Domestic Commodities",) + (x,) for x in power_suppliers]
        exported_energy = old_df.loc[power_slice, ("Trade", "Rest of World")]
        df.loc[power_slice, ("Trade", "Rest of World")] = exported_energy

        # The grid doesn't buy exported power
        df.loc[power_slice, ("Institution", "Grid")] -= exported_energy

    if to_3x3:
        from cge_modeling.gams.gams_constants import AGR_CODES, MFG_CODES, SERV_CODES

        SERV_CODES = SERV_CODES.copy()
        del SERV_CODES[SERV_CODES.index("TnD")]
        groups = ["Agriculture", "Manufacturing", "Service"]
        codes = [AGR_CODES, MFG_CODES, SERV_CODES]
        for group, codes in zip(groups, codes):
            df = merge_values(df, "Activities", codes, group)
            df = merge_values(df, "Domestic Commodities", codes, group)
            df = merge_values(df, "Imported Commodities", codes, group)
            df = merge_values(df, "Import Sales Tax", codes, group)
            df = merge_values(df, "Domestic Sales Tax", codes, group)

    return df

In [150]:
sam_grid = aggregate_SAM(SRB, to_3x3=True, add_grid=True).loc[
    reduced_group_order, reduced_group_order
]

In [141]:
# sam.to_csv('data/SRB_SAM_3x3.csv')

In [168]:
power_suppliers = [
    "CoalP",
    "GasP",
    "HydroP",
    "NuclearP",
    "OilP",
    "OtherP",
    "SolarP",
    "WindP",
    "TnD",
]
power_slice = sorted(
    [
        (f"{group} Commodities",) + (x,)
        for x in power_suppliers
        for group in ["Domestic", "Imported"]
    ]
)
sam.loc[["Domestic Commodities", "Imported Commodities"], ("Institution", "Government")]

group                 value        
Domestic Commodities  Agriculture        21.601966
                      CoalP               0.000000
                      GasP                0.000000
                      HydroP              0.000000
                      Manufacturing     131.295299
                      NuclearP            0.000000
                      OilP                0.000000
                      OtherP              0.000000
                      Service          6126.875879
                      SolarP              0.000000
                      TnD                 0.000000
                      WindP               0.000000
Imported Commodities  Agriculture         0.743774
                      CoalP               0.000000
                      GasP                0.000000
                      HydroP              0.000000
                      Manufacturing     317.474146
                      NuclearP            0.000000
                      OilP                0.00

group         Domestic Commodities                                          \
value                        CoalP GasP HydroP NuclearP OilP OtherP SolarP   
value                                                                        
Rest of World                  0.0  0.0    0.0      0.0  0.0    0.0    0.0   

group                    Imported Commodities                        \
value          TnD WindP                CoalP       GasP     HydroP   
value                                                                 
Rest of World  0.0   0.0           167.134454  79.269871  45.346441   

group                                                                           
value           NuclearP     OilP     OtherP     SolarP         TnD      WindP  
value                                                                           
Rest of World  63.812385  8.27672  25.207231  10.623319  211.661432  18.093118